In [1]:
!pwd

/workspace/code/gym-continuousDoubleAuction_agent_based_0/gym_continuousDoubleAuction


In [2]:
%run ../gym_continuousDoubleAuction/test/test_accounting.py

2025-12-21 15:53:36,239	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-12-21 15:53:37,660	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
.............
----------------------------------------------------------------------
Ran 13 tests in 0.007s

OK


In [3]:
%run ../gym_continuousDoubleAuction/test/test_orderBook_1.py

***Bids***
  size price  trade_id  timestamp  order_id
0    5    99       100          5         5
1    5    99       102          7         7
2    5    98       101          6         6
3    5    97       103          8         8
***Asks***
  size price  trade_id  timestamp  order_id
0    5   101       100          1         1
1    5   101       102          3         3
2    5   101       103          4         4
3    5   103       101          2         2
***tape***


{'type': 'limit', 'side': 'bid', 'quantity': 2, 'price': 102, 'trade_id': 109}
Trade occurs as incoming bid limit crosses best ask
[{'timestamp': 9, 'price': Decimal('101'), 'quantity': 2, 'time': 9, 'counter_party': {'ID': 100, 'side': 'ask', 'order_id': 1, 'new_book_quantity': Decimal('3')}, 'init_party': {'ID': 109, 'side': 'bid', 'order_id': None, 'new_book_quantity': None}}]
***Bids***
  size price  trade_id  timestamp  order_id
0    5    99       100          5         5
1    5    99       102          7         7

In [4]:
%run ../gym_continuousDoubleAuction/test/test_orderbook_2.py

............
----------------------------------------------------------------------
Ran 12 tests in 0.005s

OK


In [5]:
%run ../gym_continuousDoubleAuction/test/repro_orderbook_double_delete_order.py

--- Starting Reproduction Script ---
Step 1: Placing initial order at price 100...
Order created with ID: 1

Step 2: Modifying price from 100 to 101...
Success! (If you see this, the bug is fixed)


In [6]:
%run ../gym_continuousDoubleAuction/test/repro_orderbook_volume_bug.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK



--- Testing Partial Fill Volume Sync ---
Placing Ask: 10 units @ 100
Initial OrderTree.volume: 10
Initial Calculated volume: 10

Executing Partial Fill: Bid 4 units @ 100
Final OrderTree.volume: 6 <--- BUG: This should be 6
Final Calculated volume: 6
SUCCESS: Volume is in sync.


In [7]:
%run ../gym_continuousDoubleAuction/test/repro_orderbook_crossed_book.py

--- Starting Crossed Book Reproduction Script ---
Step 1: Placing limit Ask at 100...
Step 2: Placing limit Bid at 90...
Current Best Bid: 90, Best Ask: 100

Step 3: Modifying Bid price from 90 to 110 (Crossing the Ask at 100)...
Resulting Best Bid: 110
Resulting Best Ask: 100

BUG REPRODUCED: The book is CROSSED.
Reason: modify_order moved the bid to 110 but did not check for matches against the Ask at 100.


In [9]:
%run ../gym_continuousDoubleAuction/test/repro_cash_bug.py

--- Starting Cash Handling Bug Reproduction Script ---
Scenario: Trader B1 has a Bid at 90. They modify it to 110, which matches an Ask at 100.
We will simulate what happens if we use the existing modify_cash_transfer logic.

Step 1: B1 places Bid at 90.
B1 Cash: 1100, Cash on Hold: 900

Step 2: B1 modifies Bid from 90 to 110 (Price increases).
--- Simulating impact of current modify_cash_transfer ---
After modify_cash_transfer -> B1 Cash: 900, Cash on Hold: 1100

--- Simulating what happens when the trade actually occurs ---
Trade occurs at price 100. Value: 1000
After size_increase_cash_transfer -> B1 Cash: -100, Cash on Hold: 1100

--- Final Account State ---
B1 Final Cash: -100
B1 Final Cash on Hold: 1100
B1 Net Position: 0
B1 Account value (cash + hold + position_at_mkt): 1000

INCONSISTENCY DETECTED: Cash on Hold is still > 0 despite the order being fully filled.
Reason: modify_cash_transfer moved money to 'Hold', but the trade logic deducted from 'Cash'.
Starting cash was 2000. 

In [ ]:
# %run ../gym_continuousDoubleAuction/inspect_latest_episode.py

In [ ]:
# %run ../gym_continuousDoubleAuction/analyze_unused.py